In [54]:
#immitates the qubit

import numpy as np

def randbin(data, F):
    phi = data.const * F * data.t
    p_0 = (math.sin(phi/2)) ** 2
    #return np.random.choice([0,1], size=(1,1), p=[p_0, 1-p_0]).reshape(1)[0]
    return p_0

In [3]:
#information about the experiment

class data:
    
    const = 100000 #magnetic constant
    
    t = 0 #present time
    t_min = 3.6 * 10 ** (-9) #starts at
    t_max = 360 * 10 ** (-9) #finishes at
    
    #set the flux range
    
    flux_min = -1.2
    flux_max = 1.2 
    flux_step = 0.1
    n = 0 #a number of dots
    
    def __init__(self, present, Tmin, Tmax, Fmin, Fmax, step):
        data.t = present
        data.t_min = Tmin
        data.t_max = Tmax
        data.flux_min = Fmin
        data.flux_max = Fmax
        data.flux_step = step
        
        data.n = round((Fmax - Fmin)/step) + 1
    
    #twices the time of the previous step
    
    def step(self):
        data.t *= 2

In [4]:
#fills service arrays with initial values

def begin(data, prev, pres, init):
    for i in range(data.n):
        prev.append(1 / data.n)
        pres.append(1 / data.n)
        init.append(1 / data.n)

In [5]:
#calculates the standart deviation

import math

def stdev(array):
    
    avg = 0
    for i in range(len(array)):
        avg += array[i]
    avg /= len(array)
    
    sum = 0
    for i in range(len(array)):
        sum += (array[i] - avg) ** 2  
        
    if sum == 0:
        return len(array) / 2 #if the function gets a uniform distribution, the standart deviation is a half of the interval
    else:
        return math.sqrt(sum / len(array))

In [6]:
#if the qubit's passport is given

def prob(array, hN, pas):
    
    sigma = stdev(array)
    return math.exp( - (hN - pas) ** 2 / (math.sqrt(2) * sigma) ** 2) / (math.sqrt(2 * math.pi) * sigma)

In [77]:
#one iteration of bayesian learning algorithn

def bayes(data, prev, pres):
    
    normalization = 0 #normalization coefficient
    
    for i in range(len(prev)):
        normalization += randbin(data, flux_min + i * flux_step) * prev[i]
    
    print(normalization)
    
    for i in range(len(pres)):
        pres[i] *= randbin(data, flux_min + i * flux_step) / normalization    

In [91]:
#main

import math

t_min = 3.6 * 10 ** (-1) #starts at
t_max = 36000 * 10 ** (-1) #finishes at
flux_min = -1.2 
flux_max = 1.2 #the flux range
flux_step = 0.01

a = data(t_min, t_min, t_max, flux_min, flux_max, flux_step)

previous = []
present = []
init = []

begin(a, previous, present, init)

print(present)

for i in range(int(math.log2(t_max/t_min))):
    bayes(a, previous, present)
    a.step()

for i in range(len(present)):
    print(present[i])


[0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.004149377593360996, 0.0041493

In [38]:
#проблемы:
# 1) как запускать алгоритм, если на первом шаге распределение равномерное и среднеквадратичное отклонение = 0 (stdev)
# на первом шаге взять половину интервала Ф
# 2) знание констант для адекватной работы

#строю график для распределений, строю сигму и наношу скейлинги